In [13]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    precision_score, recall_score
)

import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


In [15]:
df1 = pd.read_csv("C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/data/one_hot_encode/train_processed_data.csv")

df2= pd.read_csv("C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/data/one_hot_encode/test_processed_data.csv")

In [16]:
xtrain=df1.drop(labels=["loan_status"],axis=1)
ytrain=df1[["loan_status"]]

In [17]:
xtest=df2.drop(labels=["loan_status"],axis=1)
ytest=df2[["loan_status"]]

In [18]:
rus = RandomUnderSampler(sampling_strategy=1.0, random_state=42)  
X_res, y_res = rus.fit_resample(xtrain, ytrain)

In [19]:
model=Sequential(
    layers=[
              Input(shape=(X_res.shape[1],)),
              Dense(64,activation="relu",kernel_regularizer=l2(0.01)),
              Dropout(0.4),
              Dense(32,activation="relu",kernel_regularizer=l2(0.01)),
              Dropout(0.3),
              Dense(16,activation="relu",kernel_regularizer=l2(0.01)),
              Dense(1,activation="sigmoid"),


    ]
)

In [20]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 64)             │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,033 (15.75 KB)

 Trainable params: 4,033 (15.75 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:

mlflow.set_tracking_uri("file:///C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/mlruns")
mlflow.set_experiment("loan_acceptance_logreg")  # same as logistic regression


with mlflow.start_run(run_name="NeuralNetwork_run"):


    model = Sequential([
        Input(shape=(xtrain.shape[1],)),
        Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
        Dropout(0.4),
        Dense(32, activation="relu", kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(16, activation="relu", kernel_regularizer=l2(0.01)),
        Dense(1, activation="sigmoid")
    ])


    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(learning_rate=0.001),
        metrics=["accuracy", "Precision", "Recall", "AUC"]
    )

    
    es = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)


    mlflow.log_params({
        "optimizer": "Adam",
        "learning_rate": 0.001,
        "loss": "binary_crossentropy",
        "l2_reg": 0.01,
        "dropout": "0.4/0.3",
        "epochs": 100,
        "patience": 5,
        "batch_size": 32
    })

   
    hist = model.fit(
        xtrain, ytrain,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[es],
        verbose=1
    )

    
    y_prob = model.predict(xtest).ravel()
    y_pred = (y_prob >= 0.5).astype(int)

    acc = accuracy_score(ytest, y_pred)
    prec = precision_score(ytest, y_pred)
    rec = recall_score(ytest, y_pred)
    auc = roc_auc_score(ytest, y_prob)


    mlflow.log_metrics({
        "test_accuracy": acc,
        "test_precision": prec,
        "test_recall": rec,
        "test_auc": auc
    })


    mlflow.keras.log_model(model, artifact_path="model")

    print(f"Model logged successfully in MLflow!")
    print(f"AUC={auc:.3f} | Precision={prec:.3f} | Recall={rec:.3f} | Accuracy={acc:.3f}")


Epoch 1/100
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - AUC: 0.8667 - Precision: 0.6782 - Recall: 0.3713 - accuracy: 0.8850 - loss: 0.4620 - val_AUC: 0.9003 - val_Precision: 0.7956 - val_Recall: 0.5101 - val_accuracy: 0.9111 - val_loss: 0.2927
Epoch 2/100
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.8949 - Precision: 0.7760 - Recall: 0.5282 - accuracy: 0.9108 - loss: 0.2889 - val_AUC: 0.9049 - val_Precision: 0.7687 - val_Recall: 0.6262 - val_accuracy: 0.9195 - val_loss: 0.2813
Epoch 3/100
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.8968 - Precision: 0.7779 - Recall: 0.5497 - accuracy: 0.9132 - loss: 0.2800 - val_AUC: 0.9046 - val_Precision: 0.7883 - val_Recall: 0.5934 - val_accuracy: 0.9190 - val_loss: 0.2707
Epoch 4/100
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.9009 - Precision: 0.7768 - Recall: 0.5599 - accuracy: 0.9141 - loss: 0.2742 - val_AUC: 0.9056 - val_Precision: 0.8938 - val_Recall: 0.4200 - val_accuracy: 0.9098 - val_loss: 0.2799
Epoch 5/100
1173/117

2025/11/12 18:44:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 18:44:41 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/12 18:44:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged successfully in MLflow!
AUC=0.926 | Precision=0.853 | Recall=0.638 | Accuracy=0.934
